https://adventofcode.com/2021/day/9

<h2>--- Day 9: Smoke Basin ---</h2><p>These caves seem to be <a href="https://en.wikipedia.org/wiki/Lava_tube" target="_blank">lava tubes</a>. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.</p>
<p>If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).</p>
<p>Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:</p>
<pre><code>2<em>1</em>9994321<em>0</em>
3987894921
98<em>5</em>6789892
8767896789
989996<em>5</em>678
</code></pre>
<p>Each number corresponds to the height of a particular location, where <code>9</code> is the highest and <code>0</code> is the lowest a location can be.</p>
<p>Your first goal is to find the <em>low points</em> - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)</p>
<p>In the above example, there are <em>four</em> low points, all highlighted: two are in the first row (a <code>1</code> and a <code>0</code>), one is in the third row (a <code>5</code>), and one is in the bottom row (also a <code>5</code>). All other locations on the heightmap have some lower adjacent location, and so are not low points.</p>
<p>The <em>risk level</em> of a low point is <em>1 plus its height</em>. In the above example, the risk levels of the low points are <code>2</code>, <code>1</code>, <code>6</code>, and <code>6</code>. The sum of the risk levels of all low points in the heightmap is therefore <code><em>15</em></code>.</p>
<p>Find all of the low points on your heightmap. <em>What is the sum of the risk levels of all low points on your heightmap?</em></p>


In [1]:
import pandas as pd 
import numpy as np

In [124]:
df = pd.read_csv("mapa.txt", sep=" ", header=None, dtype={0: str})
df.head(2)

,0
0,6546798789123567965567895323943212345892129763...
1,5435799654012459943456789519894301234679098654...


In [125]:
columnas = [x for x in range(len(df.iloc[0][0]))]
df_mapa = pd.DataFrame({}, columns=columnas)

for i in range(df.shape[0]):
    df_mapa.loc[df_mapa.shape[0]] = np.array(list(df.iloc[i][0]))
df_mapa

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,6,5,4,6,7,9,8,7,8,9,...,5,3,2,5,6,8,9,2,3,6
1,5,4,3,5,7,9,9,6,5,4,...,6,4,3,4,9,9,0,1,3,5
2,6,6,5,6,9,8,7,6,4,3,...,6,5,5,6,7,8,9,2,9,6
3,7,7,6,7,8,9,8,7,5,9,...,8,6,6,9,8,9,3,9,8,9
4,9,8,9,9,9,9,9,9,9,8,...,9,8,7,9,9,9,2,4,6,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9,8,7,5,2,1,4,5,6,7,...,6,9,8,7,6,5,7,5,6,7
96,8,7,6,5,3,2,3,4,5,6,...,5,6,9,9,7,6,8,9,7,8
97,9,8,7,6,4,3,5,6,7,7,...,3,2,2,4,9,7,8,9,8,9
98,3,9,8,7,5,7,6,7,9,8,...,6,1,0,2,9,8,9,4,9,1


In [105]:
def elementoDerecha(tabla, fila, colum):
    if colum == tabla.shape[1] - 1:
        return float("inf")
    else:
        return int(tabla.iloc[fila][colum+1])
    
def elementoIzq(tabla, fila, colum):
    if colum == 0:
        return float("inf")
    else:
        return int(tabla.iloc[fila][colum-1])
    
def elementoArriba(tabla, fila, colum):
    if fila == 0:
        return float("inf")
    else:
        return int(tabla.iloc[fila-1][colum])
    
def elementoAbajo(tabla, fila, colum):
    if fila == tabla.shape[0]-1:
        return float("inf")
    else:
        return int(tabla.iloc[fila+1][colum])
    
def esMinimoLocal(tabla, fila, colum):
    lista = np.array([elementoIzq(tabla , fila, colum),
                      elementoDerecha(tabla , fila, colum),
                      elementoArriba(tabla , fila, colum),
                      elementoAbajo(tabla , fila, colum)])
    elemento = int(tabla.iloc[fila][colum])
    if lista[lista > elemento].size == 4:
        return True
    return False

In [126]:
total = 0
coor_punt_min = []
for i in range(df_mapa.shape[0]):
    for j in range(df_mapa.shape[1]):
        if esMinimoLocal(df_mapa, i, j):
            #print(df_mapa.iloc[i][j])
            coor_punt_min.append((i, j))
            total+= int(df_mapa.iloc[i][j])+1
total

491

#### resultado 491

<h2 id="part2">--- Part Two ---</h2><p>Next, you need to find the largest basins so you know what areas are most important to avoid.</p>
<p>A <em>basin</em> is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height <code>9</code> do not count as being in any basin, and all other locations will always be part of exactly one basin.</p>
<p>The <em>size</em> of a basin is the number of locations within the basin, including the low point. The example above has four basins.</p>
<p>The top-left basin, size <code>3</code>:</p>
<pre><code><em>21</em>99943210
<em>3</em>987894921
9856789892
8767896789
9899965678
</code></pre>
<p>The top-right basin, size <code>9</code>:</p>
<pre><code>21999<em>43210</em>
398789<em>4</em>9<em>21</em>
985678989<em>2</em>
8767896789
9899965678
</code></pre>
<p>The middle basin, size <code>14</code>:</p>
<pre><code>2199943210
39<em>878</em>94921
9<em>85678</em>9892
<em>87678</em>96789
9<em>8</em>99965678
</code></pre>
<p>The bottom-right basin, size <code>9</code>:</p>
<pre><code>2199943210
3987894921
9856789<em>8</em>92
876789<em>678</em>9
98999<em>65678</em>
</code></pre>
<p>Find the three largest basins and multiply their sizes together. In the above example, this is <code>9 * 14 * 9 = <em>1134</em></code>.</p>
<p><em>What do you get if you multiply together the sizes of the three largest basins?</em></p>


In [113]:
def buscandoCuenco(tabla, fila, colum, puntos_cuenca):
    e = int(tabla.iloc[fila][colum])
    puntos_cuenca.append((fila, colum))
    
    e_der = elementoDerecha(tabla , fila, colum)
    if e < e_der and e_der not in [9, float("inf")] and (fila, colum+1) not in puntos_cuenca:
        buscandoCuenco(tabla, fila, colum+1, puntos_cuenca)
        
    e_izq = elementoIzq(tabla , fila, colum)
    if e < e_izq and e_izq not in [9, float("inf")] and (fila, colum-1) not in puntos_cuenca:
        buscandoCuenco(tabla, fila, colum-1, puntos_cuenca)
        
    e_arri = elementoArriba(tabla , fila, colum)
    if e < e_arri and e_arri not in [9, float("inf")] and (fila-1, colum) not in puntos_cuenca:
        buscandoCuenco(tabla, fila-1, colum, puntos_cuenca)
        
    e_aba = elementoAbajo(tabla , fila, colum)
    if e < e_aba and e_aba not in [9, float("inf")] and (fila+1, colum) not in puntos_cuenca:
        buscandoCuenco(tabla, fila+1, colum, puntos_cuenca)

In [127]:
coor_punt_min[:10]

[(0, 46),
 (0, 92),
 (1, 2),
 (1, 10),
 (1, 26),
 (1, 32),
 (1, 40),
 (1, 54),
 (1, 66),
 (1, 96)]

In [132]:
cant_puntos_cuanca = []
for xy in coor_punt_min:
    puntos_cuenca = []
    buscandoCuenco(df_mapa, xy[0], xy[1], puntos_cuenca)
    cant_puntos_cuanca.append(len(puntos_cuenca))
cant_puntos_cuanca.sort()
cant_puntos_cuanca[-3:]

[97, 99, 112]

In [133]:
cant_puntos_cuanca[-3:][0]* cant_puntos_cuanca[-3:][1]*cant_puntos_cuanca[-3:][2]

1075536

#### resultado: 1075536